In [98]:
import numpy as np, re
from matplotlib import pyplot as plt
from functools import reduce
from copy import deepcopy

In [148]:

def step(valves,minutes=30, valve = 'AA', pressure_released =0, total_power=0, blocked_valves=[]):
    valves = reduce_paths(valves=valves, current_valve=valve)
    current_valve = valves[valve]
    minutes = max(0, minutes)
    pressure_released = total_power*(30-minutes)
    if not minutes:
        return pressure_released
    a = [
        step(
            valves=deepcopy(valves), minutes = minutes-current_valve['neibors'][v],
            valve=v,
            pressure_released=pressure_released,
            total_power=total_power,
            blocked_valves=(len(valves[v]['neibors'])>1)*[valve])
            for v in current_valve['neibors'] if v not in blocked_valves]
    a = max(a)
    if current_valve['pressure_released']:
        new_valves = deepcopy(valves)
        new_valves[valve]['pressure_released'] = 0
        b = [
            step(
                new_valves,
                minutes = minutes-current_valve['neibors'][v]-1,
                valve=v, pressure_released=pressure_released,
                total_power=total_power+current_valve['pressure_released'])
                for v in current_valve['neibors']]
        b = max(b)
        return max(a,b)
    return a

In [151]:
def reduce_paths(valves, current_valve='AA'):
    
    modified = True
    while modified:
        modified = False
        to_delete = []
        for valve in valves:
            if len(valves[valve]['neibors']) ==2 and not valves[valve]['pressure_released'] and not valve == current_valve:
                modified = True
                nbr = list(valves[valve]['neibors'].items())
                dist = nbr[0][1] + nbr[1][1]
                valves[nbr[0][0]]['neibors'][nbr[1][0]] = dist
                valves[nbr[1][0]]['neibors'][nbr[0][0]] = dist

                to_delete.append(valve)
                for v in valves:
                    try:
                        del(valves[v]['neibors'][valve])
                    except KeyError:
                        pass
            if len(valves[valve]['neibors']) ==1 and not valves[valve]['pressure_released'] and not valve == current_valve:
                modified = True
                to_delete.append(valve)
                for v in valves:
                    try:
                        del(valves[v]['neibors'][valve])
                    except KeyError:
                        pass
        # print(valves)
        for v in to_delete:
            del(valves[v])
                
    return valves

In [152]:
input = open('p16.txt', mode='r').readlines()
valves = {}
for line in input:
    if line:
        # print(line)
        parsed = re.findall(r'Valve ([A-Z][A-Z]) has flow rate=(\d+)', line)[0]
        neibors = re.findall(r'([A-Z][A-Z])', line)
        neibors = [{n: 1} for n in neibors]
        # print(neibors)
        valves[parsed[0]] = {'pressure_released':int(parsed[1]), 'neibors': reduce(lambda x,y: x|y,neibors[1:])}
# valves = reduce_paths(valves=deepcopy(valves))
step(deepcopy(valves),minutes=15)

KeyError: 'AA'

In [142]:
valves

{'AA': {'pressure_released': 0, 'neibors': {'DD': 1, 'BB': 1, 'JJ': 2}},
 'BB': {'pressure_released': 13, 'neibors': {'CC': 1, 'AA': 1}},
 'CC': {'pressure_released': 2, 'neibors': {'DD': 1, 'BB': 1}},
 'DD': {'pressure_released': 20, 'neibors': {'CC': 1, 'AA': 1, 'EE': 1}},
 'EE': {'pressure_released': 3, 'neibors': {'DD': 1, 'HH': 3}},
 'HH': {'pressure_released': 22, 'neibors': {'EE': 3}},
 'JJ': {'pressure_released': 21, 'neibors': {'AA': 2}}}